<a href="https://colab.research.google.com/github/Shahad-UOL/Project/blob/main/%5BFinal_version_submit%5D_Project_Code_Shahad_Alghamdi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run the below cell first

In [ ]:
pip install pyldavis


# Download the Data

In [ ]:
# This is the basic steps performed to merge both data and remove unessarly info

import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import numpy as np


In [ ]:
def read_and_merge(file1, file2):
  
  #Read file and store in df
  meta_data=[]
  with gzip.open(file1) as f:
      for l in f:
          meta_data.append(json.loads(l.strip()))

  df_meta_data = pd.DataFrame.from_dict(meta_data)

  #Read file and store in df
  review_data=[]
  with gzip.open(file2) as f:
      for l in f:
          review_data.append(json.loads(l.strip()))

  df_reviews = pd.DataFrame.from_dict(review_data)

  #Merge dfs
  df_merged = pd.merge(df_reviews, df_meta_data, on=['asin','asin'])
  return df_merged



In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Arts_Crafts_and_Sewing.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz

df_Arts= read_and_merge("meta_Arts_Crafts_and_Sewing.json.gz","Arts_Crafts_and_Sewing.json.gz")


In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Video_Games.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Video_Games.json.gz

df_Video_Games= read_and_merge("meta_Video_Games.json.gz","Video_Games.json.gz")


In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Grocery_and_Gourmet_Food.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Grocery_and_Gourmet_Food.json.gz

df_Grocery= read_and_merge("meta_Grocery_and_Gourmet_Food.json.gz","Grocery_and_Gourmet_Food.json.gz")


In [ ]:
#Cleaning
def data_cleaning(df_merged,file_name,vid=False):
  #Drop columns
  df_merged=df_merged.drop(columns=['reviewTime','overall','category','reviewerID', 'style','reviewerName', 'unixReviewTime','image','summary','verified','vote','image', 'tech1','description', 'fit','tech2', 'rank', 'details', 'main_cat', 'date', 'price', 'imageURL', 'imageURLHighRes','also_buy','feature','also_view','similar_item'])

  #Remove Duplicates
  df_merged=df_merged.dropna(how='any',axis=0) 

  #Replace empty cell with Nan, Then drop all Nan
  #Do this for 'brand' and 'title' and 'reviewText'
  df_merged['brand'].replace('', np.nan, inplace=True)
  df_merged.dropna(subset=['brand'], inplace=True)

  df_merged['title'].replace('', np.nan, inplace=True)
  df_merged.dropna(subset=['title'], inplace=True)

  df_merged['reviewText'].replace('', np.nan, inplace=True)
  df_merged.dropna(subset=['reviewText'], inplace=True)

  #Remove duplicate reviews
  df_merged=df_merged.drop_duplicates(subset=['reviewText'])

  #For video category
  if (vid==True):
      df_merged['brand']=df_merged['brand'].str.replace(r'by\n    \n    ','')

  #write to csv
  #Also save to your drive for later analysis
  df_merged.to_csv(file_name+'.csv')
  return df_merged

In [ ]:
df_Arts=data_cleaning(df_Arts,'Arts_merged_noDuplicate',vid=True)

In [ ]:
df_Video_Games=data_cleaning(df_Video_Games,'Video_Games_merged_noDuplicate',vid=True)

In [ ]:
df_Grocery=data_cleaning(df_Grocery,'Grocery_merged_noDuplicate',vid=True)

## Select the data

In [ ]:
def select_products(df, column, value, file_name):

  final_df =df.loc[df[column] == value]
  final_df=final_df.reset_index()
  final_df.insert(0, 'ID', final_df.index + 1)
  final_df= final_df.drop(columns=['index'])
  final_df.to_csv(file_name+'.csv')
  return final_df

In [ ]:
#Select Art subsets
#Brother Sewing Machine
df_Art_Product_A=select_products(df_Video_Games,'asin','B000F7DPEQ','Art_Product_A')

#SINGER Sewing Machine
df_Art_Product_B =select_products(df_Video_Games,'asin','B005HR1JMO','Art_Product_B')


In [ ]:
#Select Video Games subsets

#Redragon Mouse
df_VideoGame_Product_A=select_products(df_Video_Games,'asin','B00HTK1NCS','Games_Product_A')

#Havit Mouse
df_VideoGame_Product_B =select_products(df_Video_Games,'asin','B00KKAQYXM','Games_Product_B')


In [ ]:
#Select Grocery subsets

#Finding all coconut oil products
df_oil_products=df_Grocery[df_Grocery['title'].str.contains("Coconut Oil",na=False, case=False)]

# we look at the top 20 products and manually check them and find which are sustainable and which are not.
# to check https://www.amazon.com/dp/B00DS842HS
df_oil_products.groupby('asin').count().sort_values('brand',ascending=False).head(20)

asin_list_not_sustainable=['B00CPZPYLS','B008MOMYRI','B00HNTPEWU','B00HNTPF7E','B01D19RF3Q','B00M8HPGMU','B01DKX9CIE','B00US7LXUQ','B00A2A88ZW']
asin_list_sustainable=['B00DS842HS','B000H2XXRS','B003OGKCDC','B000GAT6NG','B00Y8HZS1W','B00KRFLDBS','B002VLZ8D0','B001E8DHPW','B000R7YVF6','B0013OX8II']

df_not_sustainable_oil=df_oil_products.loc[df_oil_products['asin'].isin(asin_list_not_sustainable)].sort_values('brand',ascending=False)
df_sustainable_oil=df_oil_products.loc[df_oil_products['asin'].isin(asin_list_sustainable)].sort_values('brand',ascending=False)#

df_not_sustainable_oil.to_csv('Grocery_not_sustainable_oil.csv')
df_sustainable_oil.to_csv('Grocery_sustainable_oil.csv')

# Pre-Processsing

##Define finctions

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models   # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import nltk; nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc),min_len=3) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts,bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Build the bigram and trigram models
def build_Bi_Tri_models(data_words):
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
  
  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)
  return bigram_mod, trigram_mod

In [ ]:
def preprocess(df,isTrigram,extra_stopwords_list):
  data = df.reviewText.values.tolist()
  # Remove Emails
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

  # Remove new line characters
  data = [re.sub('\s+', ' ', sent) for sent in data]

  # Remove distracting single quotes
  data = [re.sub("\'", "", sent) for sent in data]

  #Convert to words
  data_words= list(sent_to_words(data))

  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases. 
  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)

  # Remove Stop Words
  stop_words.extend(extra_stopwords_list)
  data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)
  
  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized_Bi = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

  return_data_processed= data_lemmatized_Bi
  
  if (isTrigram==True):
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    ## Form Trigrams
    data_words_bigrams_trigrams = make_trigrams(data_words_bigrams, bigram_mod,trigram_mod)
    data_lemmatized_Tri = lemmatization(data_words_bigrams_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return_data_processed = data_lemmatized_Tri


  return return_data_processed ,data

In [ ]:
#Creat Corpus and id2word
def creat_corpus(data_processed,filter_extremes):
  
  id2word = corpora.Dictionary(data_processed)
  id2word.filter_extremes(no_below=filter_extremes, no_above=0.5, keep_n=100000)
  # Create Corpus
  texts = data_processed
  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]
  

  return id2word, corpus

##Preprocess each subset

In [ ]:
#Arts

#Custom stopwords
extra_stopwords_list=['brother', 'sew', 'machine']

#Borther Sewing Machine subset
data_processed_Art_A, data_Art_A= preprocess(df=df_Art_Product_A,isTrigram=False,extra_stopwords_list=extra_stopwords_list)
id2word_Art_A, corpus_Art_A = creat_corpus(data_processed,5)

extra_stopwords_list=['Singer', 'sew', 'machine']
#SINGER Sewing Machine subset
data_processed_Art_B, data_Art_B= preprocess(df=df_Art_Product_B,isTrigram=True,extra_stopwords_list=extra_stopwords_list)
id2word_Art_B, corpus_Art_B = creat_corpus(data_processed,10)

In [ ]:
#Video Games

#Custom stopwords
extra_stopwords_list=['use','great','good','get','try','nice']

#Redragin Gaming Mouse subset
data_processed_VideoGames_A, data_VideoGames_A = preprocess(df=df_VideoGame_Product_A,isTrigram=True,extra_stopwords_list=extra_stopwords_list)
id2word_VideoGames_A, corpus_VideoGames_A = creat_corpus(data_processed,10)

#Havit Gaming Mouse subset
data_processed_VideoGames_B, data_VideoGames_B= preprocess(df=df_VideoGame_Product_B,isTrigram=False,extra_stopwords_list=extra_stopwords_list)
id2word_VideoGames_B, corpus_VideoGames_B = creat_corpus(data_processed,10)


In [ ]:
#Grocery

#Custom stopwords
extra_stopwords_list=['oil', 'product', 'coconut', 'great', 'love', 'good']

#Sustainable coconut oils subset
data_processed_Grocery_A, data_Grocery_A= preprocess(df=df_sustainable_oil,isTrigram=True,extra_stopwords_list=extra_stopwords_list)
id2word_Grocery_A, corpus_Grocery_A = creat_corpus(data_processed,15)

extra_stopwords_list=['oil', 'product', 'coconut', 'great', 'love', 'good', 'use']
#Unsustainable coconut oils subset
data_processed_Grocery_B,data_Grocery_B= preprocess(df=df_not_sustainable_oil,isTrigram=False,extra_stopwords_list=extra_stopwords_list)
id2word_Grocery_B, corpus_Grocery_B = creat_corpus(data_processed,15)

# Topic Modeling 

##Build LDA Model

In [ ]:
def compute_coherence_values( dictionary, corpus, texts, limit, chunksize=2000, passes=20, start=2, step=1 ):
    
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """

    log_perplexity_values=[]
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        

    return model_list, coherence_values

In [ ]:
#Plot graphs and scores
def show_graph(coherence_values, title_1, limit=15,start=2, step=1):
  #limit=15;start=2; step=1;
  x = range(start, limit, step)
  plt.plot(x, coherence_values, label='Coherence score')
  plt.xlabel("Topics")
  plt.ylabel("Coherence score")
  plt.title(title_1)
  plt.show()  

  for m, cv in zip(x, coherence_values ):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
  print('\n')


##The optimal model settings for each subset

In this section, we only run the below cells multiple times to find the best expriment. but for illustration, we set one run with best parameters and settings. 

####Arts

In [ ]:
#Brother
# may take long time to finish running
model_list_prod_A, coherence_values_prod_A = compute_coherence_values(dictionary=id2word_Art_A, corpus=corpus_Art_A, texts=data_processed_Art_A,chunksize=1000,passes=20, start=2, limit=10, step=1)

# Plot the results
title_1='Coherence scores for Brother Sewing Machine models (Bigram)'
show_graph(coherence_values_prod_A, title_1, limit=10,start=2, step=1)

# Choose the optimal model based on the coherence and print top 15 words
optimal_model_Art_A = model_list_prod_A[5]
pprint(optimal_model_Art_A.print_topics(num_words=15))

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model_Art_A, corpus_Art_A, id2word_Art_A)
vis


In [ ]:
#Brother
# may take long time to finish running
model_list_prod_B, coherence_values_prod_B = compute_coherence_values(dictionary=id2word_Art_B, corpus=corpus_Art_B, texts=data_processed_Art_B,chunksize=2000,passes=20, start=2, limit=10, step=1)

# Plot the results
title_1='Coherence scores for SINGER Sewing Machine models (Trigram)'
show_graph(coherence_values_prod_B, title_1, limit=10,start=2, step=1)

# Choose the optimal model based on the coherence and print top 15 words
optimal_model_Art_B = model_list_prod_B[3]
pprint(optimal_model_Art_B.print_topics(num_words=15))

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model_Art_B, corpus_Art_B, id2word_Art_B)
vis

####Video Games

In [ ]:
#Redragon Gaming Mouse
# may take long time to finish running
model_list_prod_A, coherence_values_prod_A = compute_coherence_values(dictionary=id2word_VideoGames_A, corpus=corpus_VideoGames_A, texts=data_processed_VideoGames_A,chunksize=2000,passes=20, start=2, limit=15, step=1)

# Plot the results
title_1='Coherence scores for Redragon Gaming Mouse models (Trigram)'
show_graph(coherence_values_prod_A, title_1, limit=15,start=2, step=1)

# Choose the optimal model based on the coherence and print top 15 words
optimal_model_VideoGames_A = model_list_prod_A[6]
pprint(optimal_model_VideoGames_A.print_topics(num_words=15))

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model_VideoGames_A, corpus_VideoGames_A, id2word_VideoGames_A)
vis


In [ ]:
#Habvit Gaming Mouse
# may take long time to finish running
model_list_prod_B, coherence_values_prod_B = compute_coherence_values(dictionary=id2word_VideoGames_B, corpus=corpus_VideoGames_B, texts=data_processed_VideoGames_B,chunksize=2000,passes=20, start=2, limit=15, step=1)

# Plot the results
title_1='Coherence scores for Habvit Gaming Mouse models (Bigram)'
show_graph(coherence_values_prod_B, title_1, limit=15,start=2, step=1)

# Choose the optimal model based on the coherence and print top 15 words
optimal_model_VideoGames_B = model_list_prod_B[3]
pprint(optimal_model_VideoGames_B.print_topics(num_words=15))

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model_VideoGames_B, corpus_VideoGames_B, id2word_VideoGames_B)
vis

####Grocery

In [ ]:
#Sustainable coconut oils
# may take long time to finish running
model_list_prod_A, coherence_values_prod_A = compute_coherence_values(dictionary=id2word_Grocery_A, corpus=corpus_Grocery_A, texts=data_processed_Grocery_A,chunksize=20000,passes=30, start=2, limit=30, step=1)

# Plot the results
title_1='Coherence scores for unsustainable products models (Trigram)'
show_graph(coherence_values_prod_A, title_1, limit=30,start=2, step=1)

# Choose the optimal model based on the coherence and print top 15 words
optimal_model_Grocery_A = model_list_prod_A[7]
pprint(optimal_model_Grocery_A.print_topics(num_words=15))

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model_Grocery_A, corpus_Grocery_A, id2word_Grocery_A)
vis

In [ ]:
#Unsustainable coconut oils
# may take long time to finish running
model_list_prod_B, coherence_values_prod_B = compute_coherence_values(dictionary=id2word_Grocery_B, corpus=corpus_Grocery_B, texts=data_processed_Grocery_B,chunksize=10000,passes=20, start=2, limit=25, step=1)

# Plot the results
title_1='Coherence scores for unsustainable products models (Bigram)'
show_graph(coherence_values_prod_B, title_1, limit=25,start=2, step=1)

# Choose the optimal model based on the coherence and print top 15 words
optimal_model_Grocery_B = model_list_prod_B[8]
pprint(optimal_model_Grocery_B.print_topics(num_words=15))

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model_Grocery_B, corpus_Grocery_B, id2word_Grocery_B)
vis

In [ ]:
#Save the model
#optimal_model.save('model_name.model')

#Load if there is saved models
#optimal_model = gensim.models.ldamodel.LdaModel.load('model_name.model')


##Find the dominant topic in each sentence

In [ ]:
def find_dominant_topic(ldamodel, corpus, texts):
  # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
      row = sorted(row[0], key=lambda x: (x[1]), reverse=True) # This for gensim lda model
      # Get the Dominant topic, Perc Contribution and Keywords for each document
      for j, (topic_num, prop_topic) in enumerate(row):
          if j == 0: # => dominant topic
              wp = ldamodel.show_topic(topic_num)
              topic_keywords = ", ".join([word for word, prop in wp])             
              sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
          else:
              break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original review to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

###Arts

In [ ]:
#Brother sewing machine
df_topic_keywords = find_dominant_topic(ldamodel=optimal_model_Art_A, corpus=corpus_Art_A, texts=data_Art_A)

# Format
df_review_topic_Art_A = df_topic_keywords
df_review_topic_Art_A.columns = [ 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

#SINGER sewing machine
df_topic_keywords = find_dominant_topic(ldamodel=optimal_model_Art_B, corpus=corpus_Art_B, texts=data_Art_B)

# Format
df_review_topic_Art_B = df_topic_keywords
df_review_topic_Art_B.columns = [ 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

###Video Games

In [ ]:
#Redragon gaming mouse
df_topic_keywords = find_dominant_topic(ldamodel=optimal_model_VideoGames_A, corpus=corpus_VideoGames_A, texts=data_VideoGames_A)

# Format
df_review_topic_VideoGames_A= df_topic_keywords
df_review_topic_VideoGames_A.columns = [ 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

#Havit gaming mouse
df_topic_keywords = find_dominant_topic(ldamodel=optimal_model_VideoGames_B, corpus=corpus_VideoGames_B, texts=data_VideoGames_B)

# Format
df_review_topic_VideoGames_B = df_topic_keywords
df_review_topic_VideoGames_B.columns = [ 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

###Grocery

In [ ]:
#Sustainable coconut oils
df_topic_keywords = find_dominant_topic(ldamodel=optimal_model_Grocery_A, corpus=corpus_Grocery_A, texts=data_Grocery_A)

# Format
df_review_topic_Grocery_A= df_topic_keywords
df_review_topic_Grocery_A.columns = [ 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

#Unsustainable coconut oils
df_topic_keywords = find_dominant_topic(ldamodel=optimal_model_Grocery_B, corpus=corpus_Grocery_B, texts=data_Grocery_B)

# Format
df_review_topic_Grocery_B = df_topic_keywords
df_review_topic_Grocery_B.columns = [ 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Sentiment Analysis

In [ ]:
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
# Calculate sentiment score 
def sentimentScore(df):
    #get reviews only
    sentences = df['Text']
    #initialise the vader object
    analyzer = SentimentIntensityAnalyzer()
    results = []
    #pass each review to the function
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        #print(str(vs))
        results.append(vs)

    #concat the sentiment score to the original df
    sentiment_df = pd.DataFrame(results) #convert list to df
    df.index = sentiment_df.index #match index
    sentiment_and_Topics = pd.concat([df, sentiment_df], axis=1) #concat both df 

    ## Convert sentiment score to text
    compound = sentiment_and_Topics['compound']
    sentiment=[]
    for i in range(len(compound)):
      if compound[i] >= 0.05 :
            sentiment.append("Positive")
    
      elif compound[i] <= - 0.05 :
            sentiment.append("Negative")
    
      else :
            sentiment.append("Neutral")

    ## produce the final DF
    overall_sentiment=pd.DataFrame(sentiment)
    sentiment_and_Topics = pd.concat([sentiment_and_Topics, overall_sentiment], axis=1)
    sentiment_and_Topics.columns = [*sentiment_and_Topics.columns[:-1], 'overall_sentiment']
    
    
    return sentiment_and_Topics


In [ ]:
#Sentiment for Art products
df_final_Art_Product_A= sentimentScore(df_review_topic_Art_A)
df_final_Art_Product_B= sentimentScore(df_review_topic_Art_B)

In [ ]:
#Sentiment for Video Games products
df_final_VideoGame_Product_A= sentimentScore(df_review_topic_VideoGames_A)
df_final_VideoGame_Product_B= sentimentScore(df_review_topic_VideoGames_B)

In [ ]:
#Sentiment for Grocery products
df_final_Grocery_Group_A= sentimentScore(df_sustainable_oil)
df_final_Grocery_Group_B= sentimentScore(df_not_sustainable_oil)

In [ ]:
#Save for analysis and visulization
#Product_with_sentiment_and_Topics.to_csv('Product_with_sentiment_and_Topics.csv')

# Visualisation

In [ ]:
import pandas as pd
from matplotlib import ticker
import matplotlib.pyplot as plt
import numpy as np
import string
import matplotlib.colors as mcolors

In [ ]:
def draw_overall_pie_2(df, topics_list,title):
  
  y = df.Total.to_list()
  colors = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','darkolivegreen','tan','darkmagenta']

  patches, texts = plt.pie(y, startangle=90, radius=1.2,colors=colors)
  labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(topics_list, df.Percentage.to_list())]

  sort_legend = True
  if sort_legend:
      patches, labels, dummy =  zip(*sorted(zip(patches, labels, y),
                                            key=lambda x: x[2],
                                            reverse=True))

  plt.legend(patches, labels, loc='left center', bbox_to_anchor=(-0.1, 1.),
            fontsize=12)
  plt.xlabel(title,labelpad=20,fontsize=14)

#plt.savefig('piechart.png', bbox_inches='tight')


In [ ]:
def draw_overall_pie(df,topics_list,title):
  
  values = df.Total.to_list()

  my_labels = topics_list
  pie=plt.pie(values,autopct='%1.2f%%',textprops={'fontsize': 14})
  #plt.title("Brother Sewing Machine Topics (Arts Crafts)",fontsize=14, y=1.08)
  plt.xlabel(title,labelpad=20,fontsize=14)
  plt.legend(pie[0],my_labels, bbox_to_anchor=(.90,0.5), loc="center right", fontsize=10, 
            bbox_transform=plt.gcf().transFigure)
  plt.subplots_adjust(left=0.0, bottom=0.1, right=0.50)

  plt.axis('equal')
  plt.show()





In [ ]:

def draw_pie(df, topic_id):
  
  title = '"'+df.index[topic_id] +'" with '+ str(df['Total'].iloc[topic_id])+' Reviews'
  values = df.iloc[topic_id, 0:3].to_list()

  my_labels = 'Positive','Neutral','Negative'
  plt.pie(values,labels=my_labels,autopct='%1.2f%%',textprops={'fontsize': 14})
  plt.title(title,fontsize=14 )
  plt.axis('equal')
  plt.show()

In [ ]:
def creat_df(df ,topics_list ):

  N_topics = df['Dominant_Topic'].unique()
  N_topics=sorted(N_topics)

  positive= df[df['overall_sentiment']=='Positive'].groupby('Dominant_Topic').count()['Text']
  Neutral= df[df['overall_sentiment']=='Neutral'].groupby('Dominant_Topic').count()['Text']
  Negative= df[df['overall_sentiment']=='Negative'].groupby('Dominant_Topic').count()['Text']
  positive=list(positive)
  Neutral=list(Neutral)
  Negative=list(Negative)
  
  df = pd.DataFrame(positive, columns=['Positive'], index=topics_list)
  #df['Positive'] = positive
  df['Neutral'] = Neutral
  df['Negative'] = Negative
  df['Total'] = df.sum(numeric_only=True,axis=1)
  df['Percentage'] = df.Total / df.Total.sum() *100

  return df

In [ ]:
def show_perc_graph(df , title ,topics_list):


  df_1= creat_df(df,topics_list)
  print(df_1)
  
  #pie for sentiment in each topic
  for i in range(0,len(topics_list)):
    draw_pie(df_1,i)
  #pie of all topics
  draw_overall_pie(df_1,topics_list,title)
  #pie of all topics version 2
  draw_overall_pie_2(df_1,topics_list,title)

  
  fig,ax= plt.subplots()
  df_1[['Positive','Neutral','Negative']].plot(
  #x = index, 
  ax=ax,
  kind = 'barh', 
  stacked = True, 
  figsize=(15,7)
  #title = "Count of CR over Topics for "+title)
  )
  
  plt.xlabel('Count of CR',fontsize=13)
  plt.ylabel("Topics",fontsize=13)
  plt.title("Count of CR over Topics for "+title, fontsize=15)
  plt.yticks(fontsize=14)
  plt.xticks(fontsize=10)
  #Label the bars
  for i, v in enumerate(df_1.Total):
    plt.text(v+0.2, i, str(round(v, 2)), va="center", fontsize=15)
  
  # use axvline to draw the mean of reviews
  mean = df_1["Total"].mean()
  ax.axvline(mean,color='red',linewidth=1,ls='--',label='μ='+str(round(mean)))
  plt.legend(fontsize=14)
 

  
  

##Art

In [ ]:
topics_list=['Multiuse','Functionality','Price','Clarity of\n Instructions','Ease of Use','Customer Service','Thread Tension']
show_perc_graph(df_final_Art_Product_A, 'Brother Sewing Machine (Arts and Crafts)',topics_list)

In [ ]:
topics_list=['Price','Thread Tension','Functionality','Customer Service','Buy as Gift','Ease of Use']
show_perc_graph(df_final_Art_Product_B, 'SINGER Sewing Machine (Arts and Crafts)',topics_list)

##Video Games

In [ ]:
topics_list=['DPI Settings','Mouse Buttons','Quality','Backlit Feature','Software Compatibility','Ergonomic Design','Price','Product Lifespan']
show_perc_graph(df_final_VideoGame_Product_A, 'Redragon Gaming Mouse (Video Games)',topics_list)

In [ ]:
topics_list=['Scroll Wheel','Colour Change','Mouse Buttons','Cord','Ergonomic Design']
show_perc_graph(df_final_VideoGame_Product_B, 'Havit Gaming Mouse (Video Games)',topics_list)

##Grocery

In [ ]:
topics_list=['Packaging','Customers experiences\nwith various brands','Texture','Price and Quality','Benefit for Skin','Smell','Flavour and Cooking','Function as Hair Product','Delivery Experience']
show_perc_graph(df_final_Grocery_Group_A, 'Sustainable Coconut Oils Products\n(Grocery and Gourmet Food)',topics_list)

In [ ]:
topics_list=['Packaging','Used with Popcorn','Time Frame','Used for Cooking','Falvour','Versatility','Added to Coffee','Used for Beauty','Price and Quality','Health Benefits','Smell','Delivery Experience','Recommend the Product']
show_perc_graph(df_final_Grocery_Group_B, 'Unsustainable Coconut Oils Products\n(Grocery and Gourmet Food)',topics_list)

# Conclusion

In [ ]:
# Parts of this code has been modified from machinelearningplus.com, the liscence below:

# All Rights Reserved. This notebook is proprietary content of machinelearningplus.com. 
# This can be shared solely for educational purposes, with due credits to machinelearningplus.com
